<a href="https://colab.research.google.com/github/Olfeng-xalaz/SCM/blob/main/Fallstudie_Juicy_AG-Erweiterung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design)

In [1]:
! pip install -q pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 49.9 MB/s eta 0:00:00


In [2]:

import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [3]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

Cloning into 'WS25_Supply_Chain_Optimierung'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 149 (delta 72), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 176.29 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [4]:
! ls

sample_data  WS25_Supply_Chain_Optimierung


In [5]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [6]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [7]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [8]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [9]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [10]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [11]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [12]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [13]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


# Erweiterung

In [14]:
!ls
!ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign

sample_data  WS25_Supply_Chain_Optimierung
NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [15]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

beschaffung = pd.read_csv(f"{folder}/NetworkDesign_Beschaffungskosten.csv",sep=";")

beschaffung.head()


,Beschaffungsmarkt,Vorprodukt,Beschaffungskosten,Kapazitäten
0,Afrika,Flasche,10,60
1,Afrika,Vitaminwasser,20,30
2,Asien,Flasche,12,100
3,Asien,Vitaminwasser,15,40
4,Europa,Flasche,15,60


# Modell für JUICY AG

In [16]:

# Erstellen einer Modellinstanz
model = Model()

# Indexmengen

In [17]:
I = list(kapazitaeten.Produktionsstandort.unique())
J = list(nachfrage.Markt.unique())
A = list(kapazitaeten.Ausbaustufe.unique())
M = list(beschaffung.Beschaffungsmarkt.unique())
P = list(beschaffung.Vorprodukt.unique())

# Entscheidungsvariablen

In [18]:
x = {}
for i in I:
  for j in J:
    x[i,j] = model.addVar(vtype='I', name = f"Produktionsmenge am Standort {i} für Markt {j}")

y = {}
for i in I:
  for a in A:
    y[i,a] = model.addVar(vtype='B', name = f"Wenn Werk {i} eine Ausbaustufe {a} baut")

b = {}
for p in P:
  for m in M:
    for i in I:
       b[p,m,i] = model.addVar(vtype='I', name = f"Beschaffungsmenge der Vorprodukte {p} am Beschaffungsmarkt {m} zum Produktionswerk {i}")

len(model.getVars()) # Zähler, Prüft die Anzahl der Variablen


85

# Parameter

In [19]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]
print("Produktionskosten:", cp)

#Der Code verwandelt eine Tabelle in einen zweidimensionalen Parameter der direkt im Optimierungsmodell verwendet werden kann. MultiIndex → Tuple-Schlüssel → Modellparamete
#Warum nicht wie im Beispiel davor? Du kannst das so machen – aber nur, wenn dein Parameter eindimensional ist. Hier brauchst du zweidimensionale Parameter.

# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]
print("Investitionen:", cf)


# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
print("Kapazitäten:", cap)


# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
print("Nachfragemenge:", d)

# Beschaffungskosten der Vorprodukte am Markt
bko = beschaffung.set_index(["Vorprodukt","Beschaffungsmarkt"]) # steht für p (Vorprodukt), m (markt)
bko = bko.to_dict("dict")["Beschaffungskosten"]
print("Beschaffungskosten:", bko)

# Kapazitäteten des Vorprodukts am Markt
capa = beschaffung.set_index(["Vorprodukt","Beschaffungsmarkt"])
capa = capa.to_dict("dict")["Kapazitäten"]
print("Kapazitäten:", capa)

Produktionskosten: {('Charleston', 'USA'): 81, ('Charleston', 'Südamerika'): 92, ('Charleston', 'Europa'): 101, ('Charleston', 'Asien'): 130, ('Charleston', 'Afrika'): 115, ('Curitiba', 'USA'): 117, ('Curitiba', 'Südamerika'): 77, ('Curitiba', 'Europa'): 108, ('Curitiba', 'Asien'): 98, ('Curitiba', 'Afrika'): 100, ('Hamburg', 'USA'): 102, ('Hamburg', 'Südamerika'): 105, ('Hamburg', 'Europa'): 95, ('Hamburg', 'Asien'): 119, ('Hamburg', 'Afrika'): 111, ('Quanzhou', 'USA'): 115, ('Quanzhou', 'Südamerika'): 125, ('Quanzhou', 'Europa'): 90, ('Quanzhou', 'Asien'): 59, ('Quanzhou', 'Afrika'): 74, ('East London', 'USA'): 142, ('East London', 'Südamerika'): 100, ('East London', 'Europa'): 103, ('East London', 'Asien'): 105, ('East London', 'Afrika'): 71}
Investitionen: {('Charleston', 'klein'): 6000, ('Charleston', 'groß'): 9000, ('Curitiba', 'klein'): 4500, ('Curitiba', 'groß'): 6750, ('Hamburg', 'klein'): 6500, ('Hamburg', 'groß'): 9750, ('Quanzhou', 'klein'): 4100, ('Quanzhou', 'groß'): 6150

# Zielfunktion

In [20]:
model.setObjective(quicksum(cp[i,j] * x[i,j] for i in I for j in J) + quicksum(cf[i,a] * y[i,a] for i in I for a in A) + quicksum(bko[p,m] * b[p,m,i] for m in M for P in p), sense = "minimize")

# Restriktionen

In [21]:
for j in J: # Produktion des Standorts muss die Nachfrage des Marktes decken
  model.addCons(quicksum(x[i,j] for i in I) >= d[j], name = f' Nachfrage von Markt {j}')

for i in I: # Produktion des Standorts darf logischerweise nicht die Kapazität der Kapazitätsstufe überschreiten
  model.addCons(quicksum(x[i,j] for j in J) <= quicksum(cap[a] * y[i,a] for a in A), name = f'Kapazität vom Standort {i}')

for i in I: # Ein Standort darf nur max. 1 Ausbaustufe besitzen
  model.addCons(quicksum(y[i,a] for a in A) <= 1, name = f'Ausbaustufe der Kapazität{a}')

for m in M: # Die Beschaffungsmenge darf die Kapazität des Beschaffungsstandorts nicht überschreiten
  for p in P:
    model.addCons(quicksum(b[p,m,i] for i in I) <= capa[p,m], name = f'Beschaffungsmenge der Vorprodukte {p} für Beschaffungsmarkt {m}')

for i in I: # Festgelegte Regel: Für jedes Produkt brauche ich jeweils eine Flasche und ein Vitaminwasser
    prod_i = quicksum(x[i,j] for j in J) # Festgelegte Produktionsmenge wieder holen
    model.addCons(quicksum(b["Flasche", m, i] for m in M) == prod_i, name = f'Flasche benötigt pro Produkt für Produktionsmenge {i}') # Für jedes Produkt der Produktionsmenge brauche ich eine Flasche
    model.addCons(quicksum(b["Vitaminwasser", m, i] for m in M) == prod_i, name = f'Vitaminwasser benötigt pro Produkt für Produktionsmenge {i}') # Für jedes Produkt der Produktionsmenge brauche ich Vitaminwasser

len(model.getConss()) # Zähler, prüft die Anzahl der Restriktionen

35

# Berechnung des Ergebnisses

In [22]:

model.optimize()
status = model.getStatus()
gap = model.getGap()
print(f"Status des Solvers: {status} , GAP={gap}% \n")

if status == 'optimal':

  print('LÖSUNG:')
  print('Zielfunktionswert (Kosten) =', model.getObjVal()) # Wie hoch sind die minimalen Gesamtkosten?
  print("\n")
  print("PRODUKTIONS-MARKT-ZUTEILUNG") # Wer produziert für wen wie viel (Standort → Markt)?
  for j in J:
    print(f"{j}:")
    for i in I:
      if model.getVal(x[i,j])>0:
        print('\t', f'{i,j} =', round(model.getVal(x[i,j])))
  print("\n")
  print("INVESTITIONEN") # Wo wird investiert (welche Ausbaustufe)?
  for i in I:
    for a in A:
      if model.getVal(y[i,a])>0:
        print(f'{i,a} =', round(model.getVal(y[i,a])))
  print("\n")
  print("Beschaffung") # Wo wird wie viel von welchen Vorprodukten für welchen Produktionsstandort beschafft?
  for m in M:
    for p in P:
      for i in I:
        if model.getVal(b[p,m,i])>0:
          print(f'{p,m,i} =', round(model.getVal(b[p,m,i])))


Status des Solvers: optimal , GAP=0.0% 

LÖSUNG:
Zielfunktionswert (Kosten) = 25077.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Curitiba', 'USA') = 12
Südamerika:
	 ('Curitiba', 'Südamerika') = 8
Europa:
	 ('Quanzhou', 'Europa') = 4
	 ('East London', 'Europa') = 10
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7


INVESTITIONEN
('Curitiba', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


Beschaffung
('Flasche', 'Afrika', 'Curitiba') = 20
('Flasche', 'Afrika', 'Quanzhou') = 20
('Flasche', 'Afrika', 'East London') = 17
('Vitaminwasser', 'Afrika', 'Quanzhou') = 20
('Vitaminwasser', 'Asien', 'Curitiba') = 20
('Vitaminwasser', 'USA', 'East London') = 17
